In [1]:
!pip install datasets transformers evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForMultipleChoice, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import evaluate
from sklearn.metrics import f1_score
import numpy as np
from torch import nn

try:
    from google.colab import drive
    drive.mount('/content/gdrive')

    train_path = '/content/gdrive/MyDrive/advanced-ml-project/data/train.tsv'
    test_path = '/content/gdrive/MyDrive/advanced-ml-project/data/test.tsv'
    dev_path = '/content/gdrive/MyDrive/advanced-ml-project/data/dev.tsv'

    eval_model_path = '/content/gdrive/MyDrive/advanced-ml-project/roberta-base-multiple-choice'

except:
    train_path = 'data/train.tsv'
    test_path = 'data/test.tsv'
    dev_path = 'data/dev.tsv'
    eval_model_path = 'roberta-base-multiple-choice'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

Mounted at /content/gdrive


device(type='cuda', index=0)

## Load Data

In [3]:
train = pd.read_csv(train_path, sep='\t', header=0)
train['label'] = train['label'].apply(lambda x: 'healthy' if x == 'not depression' else x)
train = train.sample(frac=1).reset_index(drop=True)
print('Length of train:', len(train))

test = pd.read_csv(test_path, sep='\t', header=0)
test['label'] = test['label'].apply(lambda x: 'healthy' if x == 'not depression' else x)
test = test.sample(frac=1).reset_index(drop=True)
print('Length of test:', len(test))

dev = pd.read_csv(dev_path, sep='\t', header=0)
dev['label'] = dev['label'].apply(lambda x: 'healthy' if x == 'not depression' else x)
dev = dev.sample(frac=1).reset_index(drop=True)
print('Length of dev:', len(dev))

print(train.label.value_counts())
train.head(10)

Length of train: 8891
Length of test: 3245
Length of dev: 4496
moderate    6019
healthy     1971
severe       901
Name: label, dtype: int64


,PID,text,label
0,train_pid_172,i thought 2019 was going to be the end of my d...,moderate
1,train_pid_171,Fuck Holidays : I feel the loneliest around th...,moderate
2,train_pid_2913,Spending NYE alone in my car : This year has s...,moderate
3,train_pid_2664,One of my best friends committed suicide a few...,moderate
4,train_pid_750,Anyone else feel violently depressed? : [removed],moderate
5,train_pid_648,Last year was the worst : I have been struggli...,moderate
6,train_pid_5890,Just had a breakdown for the first time since ...,moderate
7,train_pid_1752,Sat in the dark and cried myself going into th...,moderate
8,train_pid_7271,Anybody wanna talk to me? I really want to tal...,healthy
9,train_pid_683,Fuck Holidays : I feel the loneliest around th...,moderate


In [4]:
# reform data
label2idx = {'moderate': 0, 'healthy': 1, 'severe':2}

train['ending0'] = ['moderate'] * len(train)
train['ending1'] = ['healthy'] * len(train)
train['ending2'] = ['severe'] * len(train)
train['prompt'] = ['The level of depression in the following tweet is'] * len(train)
train['label'] = train['label'].apply(lambda x: label2idx[x])

test['ending0'] = ['moderate'] * len(test)
test['ending1'] = ['healthy'] * len(test)
test['ending2'] = ['severe'] * len(test)
test['prompt'] = ['The level of depression in the following tweet is'] * len(test)
test['label'] = test['label'].apply(lambda x: label2idx[x])

dev['ending0'] = ['moderate'] * len(dev)
dev['ending1'] = ['healthy'] * len(dev)
dev['ending2'] = ['severe'] * len(dev)
dev['prompt'] = ['The level of depression in the following tweet is'] * len(dev)
dev['label'] = dev['label'].apply(lambda x: label2idx[x])
train.head(10)

,PID,text,label,ending0,ending1,ending2,prompt
0,train_pid_172,i thought 2019 was going to be the end of my d...,0,moderate,healthy,severe,The level of depression in the following tweet is
1,train_pid_171,Fuck Holidays : I feel the loneliest around th...,0,moderate,healthy,severe,The level of depression in the following tweet is
2,train_pid_2913,Spending NYE alone in my car : This year has s...,0,moderate,healthy,severe,The level of depression in the following tweet is
3,train_pid_2664,One of my best friends committed suicide a few...,0,moderate,healthy,severe,The level of depression in the following tweet is
4,train_pid_750,Anyone else feel violently depressed? : [removed],0,moderate,healthy,severe,The level of depression in the following tweet is
5,train_pid_648,Last year was the worst : I have been struggli...,0,moderate,healthy,severe,The level of depression in the following tweet is
6,train_pid_5890,Just had a breakdown for the first time since ...,0,moderate,healthy,severe,The level of depression in the following tweet is
7,train_pid_1752,Sat in the dark and cried myself going into th...,0,moderate,healthy,severe,The level of depression in the following tweet is
8,train_pid_7271,Anybody wanna talk to me? I really want to tal...,1,moderate,healthy,severe,The level of depression in the following tweet is
9,train_pid_683,Fuck Holidays : I feel the loneliest around th...,0,moderate,healthy,severe,The level of depression in the following tweet is


In [5]:
data = DatasetDict({
    'train': Dataset.from_pandas(train),
    'test': Dataset.from_pandas(test),
    'dev': Dataset.from_pandas(dev)
})
data

DatasetDict({
    train: Dataset({
        features: ['PID', 'text', 'label', 'ending0', 'ending1', 'ending2', 'prompt'],
        num_rows: 8891
    })
    test: Dataset({
        features: ['PID', 'text', 'label', 'ending0', 'ending1', 'ending2', 'prompt'],
        num_rows: 3245
    })
    dev: Dataset({
        features: ['PID', 'text', 'label', 'ending0', 'ending1', 'ending2', 'prompt'],
        num_rows: 4496
    })
})

## Pre-processing

In [6]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

ending_names = ["ending0", "ending1", "ending2"]

def preprocess_function(examples):
    first_sentences = [[context] * 3 for context in examples["text"]]
    question_headers = examples["prompt"]
    second_sentences = [
        [f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + 3] for i in range(0, len(v), 3)] for k, v in tokenized_examples.items()}

tokenized_data = data.map(preprocess_function, batched=True, remove_columns=['PID', 'text', 'ending0', 'ending1', 'ending2', 'prompt'])
tokenized_data

Map:   0%|          | 0/8891 [00:00<?, ? examples/s]

Map:   0%|          | 0/3245 [00:00<?, ? examples/s]

Map:   0%|          | 0/4496 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 8891
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 3245
    })
    dev: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 4496
    })
})

In [7]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

## Training

In [8]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    matrics = accuracy.compute(predictions=predictions, references=labels)
    matrics['f1'] = f1_score(labels, predictions, average='weighted')
    return matrics

In [9]:
model = AutoModelForMultipleChoice.from_pretrained("roberta-base", num_labels=3)

training_args = TrainingArguments(
    output_dir="model_output",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

def compute_class_weight(train_y):
    """
    Compute class weight given imbalanced training data
    Usually used in the neural network model to augment the loss function (weighted loss function)
    Favouring/giving more weights to the rare classes.
    """
    import sklearn.utils.class_weight as scikit_class_weight

    train_y = [each for each in train_y]
    class_list = list(set(train_y))
    class_weight_value = scikit_class_weight.compute_class_weight(class_weight='balanced', classes=class_list, y=train_y)

    return torch.tensor(class_weight_value).to(torch.float).to(device)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)

        loss_fct = nn.CrossEntropyLoss(weight=compute_class_weight(train.label.to_list()))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'classifier.weight', 'classifier.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([4, 3, 512])
torch.Size([4, 3, 512])


Epoch,Training Loss,Validation Loss


torch.Size([4, 3, 240])
torch.Size([4, 3, 151])
torch.Size([4, 3, 308])
torch.Size([4, 3, 398])
torch.Size([4, 3, 163])


KeyboardInterrupt: ignored

In [ ]:
trainer.save_model(eval_model_path)

## Evaluating

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(eval_model_path)
model = AutoModelForMultipleChoice.from_pretrained(eval_model_path)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.predict(tokenized_data['dev'])